# Importing packages

In [1]:
import os

import grid2op
import gymnasium as gym
import ray
from grid2op.gym_compat import GymEnv, ScalerAttrConverter, MultiToTupleConverter
from ray.rllib.algorithms import ppo  # import the type of agents
from typing import Any


/Users/barberademol/Documents/GitHub/mahrl_grid2op/venv_mahrl/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-11-03 09:48:38,930	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2023-11-03 09:48:39,803	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2023-11-03 09:48:40,616	WARNING deprecation.py:50 -- DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!


# Global settings

In [2]:
ENV_NAME = "rte_case5_example"
LIBRARY_DIRECTORY = "/Users/barberademol/Documents/GitHub/mahrl_grid2op/venv_mahrl/lib/python3.10/site-packages/grid2op/data/"
NB_STEP_TRAIN = 10

In [3]:
print(LIBRARY_DIRECTORY + ENV_NAME + "_train")

/Users/barberademol/Documents/GitHub/mahrl_grid2op/venv_mahrl/lib/python3.10/site-packages/grid2op/data/rte_case5_example_train


# Only run first time to set-up

In [4]:
if not os.path.exists(LIBRARY_DIRECTORY + ENV_NAME + "_train"):
    env = grid2op.make(ENV_NAME, test=True)

    # extract 10% of the "chronics" to be used in the validation environment, 10% for testing,
    # 80% for training
    nm_env_train, nm_env_val, nm_env_test = env.train_val_split_random(
        pct_val=10.0, pct_test=10.0, add_for_test="test"
    )
    # and now you can use the training set only to train your agent:
    print(f"The name of the training environment is {nm_env_train}")
    print(f"The name of the validation environment is {nm_env_val}")
    print(f"The name of the test environment is {nm_env_test}")



# Speeding up

In [5]:
# TODO: The grid2op documentation is full of details to "optimize" the number of steps you can do
# per seconds. This number can rise from a few dozen per seconds to around a thousands per seconds
# with proper care. We strongly encouraged you to leverage all the possibilities which includes
# (but are not limited to):
# - using "lightsim2grid" as a backend for a 10-15x speed up in the "env.step(...)" function
# - using "MultifolderWithCache"/"env.chronics_handler.set_chunk(...)" for faster "env.reset(...)"
#   see https://grid2op.readthedocs.io/en/latest/environment.html#optimize-the-data-pipeline
# - using "SingleEnvMultiProcess" for parrallel computation

# Define environment

In [9]:
# MyEnv class, and train a Proximal Policy Optimisation based agent
class MyEnv(gym.Env):
    """Encapsulate Grid2Op environment and set action/observation space."""

    def __init__(self, env_config):
        # 1. create the grid2op environment
        if not "env_name" in env_config:
            raise RuntimeError(
                "The configuration for RLLIB should provide the env name"
            )
        nm_env = env_config["env_name"]
        del env_config["env_name"]
        self.env_glop = grid2op.make(nm_env, **env_config)

        # 2. create the gym environment
        self.env_gym = GymEnv(self.env_glop)
        obs_gym = self.env_gym.reset()

        # 3. (optional) customize it (see section above for more information)
        # customize action space
        self.env_gym.action_space = self.env_gym.action_space.ignore_attr(
            "set_bus"
        ).ignore_attr("set_line_status")
        self.env_gym.action_space = self.env_gym.action_space.reencode_space(
            "change_bus", MultiToTupleConverter()
        )
        self.env_gym.action_space = self.env_gym.action_space.reencode_space(
            "change_line_status", MultiToTupleConverter()
        )
        ## customize observation space
        ob_space = self.env_gym.observation_space
        ob_space = ob_space.keep_only_attr(
            ["rho", "gen_p", "load_p", "topo_vect", "actual_dispatch"]
        )
        ob_space = ob_space.reencode_space(
            "actual_dispatch",
            ScalerAttrConverter(substract=0.0, divide=self.env_glop.gen_pmax),
        )
        ob_space = ob_space.reencode_space(
            "gen_p", ScalerAttrConverter(substract=0.0, divide=self.env_glop.gen_pmax)
        )
        ob_space = ob_space.reencode_space(
            "load_p",
            ScalerAttrConverter(
                substract=obs_gym[0]["load_p"], divide=0.5 * obs_gym[0]["load_p"]
            ),
        )
        self.env_gym.observation_space = ob_space

        # 4. specific to rllib
        self.action_space = self.env_gym.action_space
        self.observation_space = self.env_gym.observation_space

        # 4.to avoid other type of issues, we recommend to build the action space and observation
        # space directly from the spaces class.
        d = {k: v for k, v in self.env_gym.observation_space.spaces.items()}
        self.observation_space = gym.spaces.Dict(d)
        a = {k: v for k, v in self.env_gym.action_space.items()}
        self.action_space = gym.spaces.Dict(a)

    def reset(self, seed: int = None, options: dict[str, Any] = None):
        obs = self.env_gym.reset()
        return obs

    def step(self, action):
        print(self.env_gym.step(action))
        obs, reward, done, info = self.env_gym.step(action)
        return obs, reward, done, info
    
    def get_env(self):
        return self.env_gym

# Train agent

In [25]:
if NB_STEP_TRAIN:
    try:
        # then define a "trainer"
        trainer = ppo.PPO(
            env=MyEnv,
            config={
                "env_config": {
                    "env_name": "rte_case5_example_train",
                    "test": True,
                },  # config to pass to env class
            },
        )
        # and then train it for a given number of iteration
        for step in range(NB_STEP_TRAIN):
            trainer.train()
    finally:
        # shutdown ray
        ray.shutdown()

2023-11-03 10:06:26,482	WARNING algorithm_config.py:2578 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
2023-11-03 10:06:26,483	WARNING deprecation.py:50 -- DeprecationWarning: `algo = Algorithm(env='<class '__main__.MyEnv'>', ...)` has been deprecated. Use `algo = AlgorithmConfig().environment('<class '__main__.MyEnv'>').build()` instead. This will raise an error in the future!
/Users/barberademol/Documents/GitHub/mahrl_grid2op/venv_mahrl/lib/python3.10/site-packages/ray/rllib/algorithms/algorithm.py:484: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARN

ValueError: Traceback (most recent call last):
  File "/Users/barberademol/Documents/GitHub/mahrl_grid2op/venv_mahrl/lib/python3.10/site-packages/ray/rllib/utils/pre_checks/env.py", line 234, in check_gym_environments
    results = env.step(sampled_action)
  File "/var/folders/9t/p1jgc3cd7v70n4nh294j6kz00000gn/T/ipykernel_9885/1020910598.py", line 67, in step
ValueError: too many values to unpack (expected 4)

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Users/barberademol/Documents/GitHub/mahrl_grid2op/venv_mahrl/lib/python3.10/site-packages/ray/rllib/utils/pre_checks/env.py", line 85, in check_env
    check_gym_environments(env, AlgorithmConfig() if config is None else config)
  File "/Users/barberademol/Documents/GitHub/mahrl_grid2op/venv_mahrl/lib/python3.10/site-packages/ray/rllib/utils/pre_checks/env.py", line 237, in check_gym_environments
    raise ValueError(
ValueError: Your gymnasium.Env's `step()` method r